## Dependencies

In [1]:
from gazpacho import get, Soup
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup             
import time
import pandas as pd
import random
# import sqlite3
# from sqlite3 import Error

options = Options()
options.headless = True
browser = Firefox(options=options)

---

## Indeed Scraper

### Scraper Function

In [2]:
def indeed_scraper(job_title, jobs_per_page, search_radius, pages):
    '''
    1. Automatically uses "+" as a space delimiter when entering job_title
    
    '''
    job_title = job_title.replace(" ", "+")
    url = f"https://ca.indeed.com/jobs?q={job_title}&l=Toronto%2C+ON&limit={jobs_per_page}&radius={search_radius}&start={pages}"
    html = get(url)
    soup = Soup(html)
    jerbs_index = soup.find("td", {"id": "resultsCol"})

    jerbs_indeed = []
    for i in range(len(jerbs_index.find("div", {"class": "title"}, mode="all"))):
        list_1 = {}
        list_1["title"] = jerbs_index.find("div", {"class": "title"}, mode="all")[i].find("a").attrs["title"]
        if jerbs_index.find("span", {"class": "company"}, mode="all")[i].text == "":
            list_1["company"] = jerbs_index.find("div", {"class": "sjcl"}, mode="all")[i].find("a")[0].text
        else: 
            list_1["company"] = jerbs_index.find("span", {"class": "company"}, mode="all")[i].text
        list_1["url"] = "https://ca.indeed.com"+jerbs_index.find("div", {"class": "title"}, mode="all")[i].find("a").attrs["href"]
        time.sleep(random.randint(1, 10) / 10)
        html_jerb = get(list_1["url"])
        soup_jerb = Soup(html_jerb)
        list_1["text"] = BeautifulSoup(" ".join([i.html for i in soup_jerb.find("div", {"class": "jobsearch-jobDescriptionText"}, mode="all")])).get_text(" ").replace("\n", " ") 
        jerbs_indeed.append(list_1)
        time.sleep(random.randint(1, 10) / 10)
    return jerbs_indeed

### Scrape with Parameters

In [3]:
job_title = "data scientist"
jobs_per_page = 50
search_radius = 100
# pages = 0
pages = [0, 50, 100, 150, 200, 250, 300] # list to iterate through
pg = []
for page in pages:
    scraped = indeed_scraper(job_title, jobs_per_page, search_radius, page)
    pg += scraped

In [4]:
# exporting to csv
filename_job = job_title.replace(" ", "_").lower()
df_indeed = pd.DataFrame(pg)
# clean up duplicates
df_indeed.drop_duplicates(subset=["title", "company", "text"], keep="first", inplace=True)
df_indeed.reset_index(inplace=True, drop=True)
df_indeed.to_csv(f"indeed_{filename_job}.csv", index=False)

---
## LinkedIn

### Scraper Function

In [7]:
def linkedin_scraper(job_title, clicks_linkedin):
    job_title = job_title.replace(" ", "%20")
    url_linkedin = f"https://www.linkedin.com/jobs/search?keywords={job_title}&location=Toronto%2C%20Ontario%2C%20Canada&trk=homepage-jobseeker_jobs-search-bar_search-submit&redirect=false&position=1&pageNum=0"
    browser.get(url_linkedin)

    for _ in range(clicks_linkedin):
    # clicking to get more jobs
        browser.find_element_by_class_name("see-more-jobs").click()
        browser.implicitly_wait(random.randint(random.randint(10, 15), random.randint(16, 20))) # seconds

    html_linkedin = browser.page_source
    soup_linkedin = Soup(html_linkedin)
    
    jerbs_linkedin = []
    for i in range(len(soup_linkedin.find("li", {"class": "result-card job-result-card result-card--with-hover-state"}, mode="all"))):
        list_2 = {}
        list_2["title"] = soup_linkedin.find("li", {"class": "result-card job-result-card result-card--with-hover-state"}, mode="all")[i].find("h3", {"class":"result-card__title job-result-card__title"}).text
        list_2["company"] = soup_linkedin.find("li", {"class": "result-card job-result-card result-card--with-hover-state"}, mode="all")[i].find("h4", {"class":"result-card__subtitle job-result-card__subtitle"}).text
        list_2["url"] = soup_linkedin.find("li", {"class": "result-card job-result-card result-card--with-hover-state"}, mode="all")[i].find("a", {"class": "result-card__full-card-link"}, mode="all")[0].attrs["href"]
        browser.implicitly_wait(random.randint(random.randint(1, 5), random.randint(6, 10))) # seconds
#         time.sleep(random.randint(1, 10) / 8)
        html_jerb = get(list_2["url"])
        soup_jerb = Soup(html_jerb)
        list_2["text"] = BeautifulSoup(" ".join([i.html for i in soup_jerb.find("div", {"class": "description__text description__text--rich"}, mode="all")])).get_text(" ").replace("\n", " ")
        jerbs_linkedin.append(list_2)
        browser.implicitly_wait(random.randint(random.randint(1, 5), random.randint(6, 10))) # seconds
#         time.sleep(random.randint(1, 10) / 8)
    return jerbs_linkedin

### Scrape with Parameters

In [8]:
clicks_linkedin = 20
job_title = "Data Scientist"

linkedin_jerbs = linkedin_scraper(job_title, clicks_linkedin)

In [9]:
# exporting to csv
filename_job = job_title.replace(" ", "_").lower()
df_linkedin = pd.DataFrame(linkedin_jerbs)
df_linkedin.drop_duplicates(subset=["title", "company", "text"], keep="first", inplace=True)
df_linkedin.reset_index(inplace=True, drop=True)
df_linkedin.to_csv(f"linkedin_{filename_job}.csv", index=False)